For this notebook, please insert where there is `_FILL_` either code or logic to make this work.



# MNIST CNN Digit Recognition Network

For this problem, you will code a basic digit recognition network. The data are images which specify the digits 1 to 10 as (1, 28, 28) data - this data is black and white images. Each pixed of the image is an intensity between 0 and 255, and together the (1, 28, 28) pixel image can be visualized as a picture of a digit. The data is given to you as $\{(x^{(i)}, y^{(i)})\}_{i=1}^{N}$ where $y$ is the given label and x is the (1, 28, 28) data. This data will be gotten from `torchvision`, a repository of computer vision data and models.

Highlevel, the model and notebook goes as follows:
*   You first download the data and specify the batch size of B = 16. Each image will need to be turned from a (1, 28, 28) volume into a serious of other volumes either via convolutional layers or max pooling layers.
*   You will pass the data through several layers to built a CNN classfier. Use the hints below to get the right dimensions and figure out what the layers should be. Be careful with the loss function. Add regularization (L1 and L2) manually.

See the comments below and fill in the analysis where there is `_FILL_` specified. All asserts should pass and Test accuracy should be about 95%.






In [ ]:
!pip uninstall --yes torchtune torchaudio torchvision
!pip install torch==2.3.0 torchtext==0.18.0 torchdata==0.8.0 portalocker>=2.0.0 torchvision==0.18

In [ ]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

In [ ]:
SEED = 1
torch.manual_seed(SEED)
_FILL_ = '_FILL_'

In [ ]:
image_path = './'

# Use ToTensor
transform = transforms.Compose(_FILL_)

mnist_train_dataset = torchvision.datasets.MNIST(
    _FILL_
  )

mnist_test_dataset = torchvision.datasets.MNIST(
   _FILL_
)

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
BATCH_SIZE = 16
LR = 0.1
L1_WEIGHT = 1e-10
L2_WEIGHT = 1e-12
EPOCHS = 20
# Get the dataloader for train and test
train_dl = _FILL_
test_dl = _FILL_

In [ ]:
class CNNClassifier(nn.Module):

  def __init__(self):
    super().__init__()
    self.cnn1 = _FILL_
    self.cnn2 = _FILL_
    self.cnn3 = _FILL_
    self.linear = _FILL_

  def forward(self, x):
    # Flatten x to be of last dimension 784
    assert(x.shape == (BATCH_SIZE, 1, 28, 28))

    # Pass through cnn layer 1
    # (28, 28, 1) -> (26, 26, 32)
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 32, 26, 26))

    # Pass through max pooling to give the result shape below
    # (26, 26, 32) -> (13, 13, 32)
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 32, 13, 13))

    # Apply ReLU
    x = _FILL_

    # Pass through cnn layer 2 to give the result below
    # (13, 13, 32) -> (11, 11, 16)
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 16, 11, 11))

    # Pass through max pooling pool to give the result below
    # (11, 11, 16) -> (5, 5, 16)
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 16, 5, 5))

    # Apply rely
    x = _FILL_

    # Pass through cnn layer 3 to give the result below
    # (5, 5, 16) -> (5, 5, 1)
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 1, 5, 5))

    # Apply rely
    x = _FILL_

    # Flatten to get the result below
    # (5, 5, 1) - > (25, )
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 25))

    # Pass through linear layer to get the result below
    # (25, ) -> (16, )
    x = _FILL_
    assert(x.shape == (BATCH_SIZE, 10))

    # Return the logits
    return _FILL_

model = CNNClassifier()

In [ ]:
# Get the loss function; remember you are outputting the logits
loss_fn = _FILL_

# Set the optimizer to SGD and let the learning rate be LR
# Do not add L2 regularization; add it manually below ...
optimizer = _FILL_

torch.manual_seed(SEED)
for epoch in range(EPOCHS):
    accuracy_hist_train = 0
    auroc_hist_train = 0.0
    loss_hist_train = 0
    # Loop through the x and y pairs of data
    for x_batch, y_batch in train_dl:
        # Get he the model predictions
        y_pred = _FILL_

        # Get the loss
        loss = _FILL_

        # Add an L1 regularizaton with a weight of L1_WEIGHT to the objective
        l1_reg = _FILL_

        # Add an L2 regularization with a weight of L2_WEIGHT to the objective
        l2_reg = _FILL_

        # Add the regularizers to the objective
        loss += _FILL_

        # Get the gradients
        _FILL_

        # Add to the loss
        # Remember loss: is a mean over the batch size and we need the total sum over the number of samples in the dataset
        loss_hist_train += _FILL_

        # Update the parameters
        _FILL_

        # Zero out the gradient
        _FILL_

        # Get the number of correct predictions, do this directly
        is_correct = _FILL_

        accuracy_hist_train += is_correct
    accuracy_hist_train /= len(train_dl.dataset)
    loss_hist_train /= len(train_dl.dataset)
    print(f'Train Metrics Epoch {epoch} Loss {loss_hist_train:.4f} Accuracy {accuracy_hist_train:.4f}')

    accuracy_hist_test = 0
    loss_hist_test = 00
    # Get the average value of each metric across the test batches
    with torch.no_grad():
      accuracy_hist_test = 0
      auroc_hist_test = 0.0
      # Loop through the x and y pairs of data
      for x_batch, y_batch in test_dl:
          # Get he the model predictions
          y_batch_pred = _FILL_

          # Get the loss
          loss = _FILL_

          # Add an L1 regularizaton with a weight of L1_WEIGHT to the objective
          l1_reg = _FILL_

          # Add an L2 regularization with a weight of L2_WEIGHT to the objective
          l2_reg = _FILL_

          # Add the regularizers to the objective
          loss += _FILL_

          # Add to the loss
          # Remember loss: is a mean over the batch size and we need the total sum over the number of samples in the dataset
          loss_hist_test += _FILL_

          # Get the number of correct predictions
          is_correct = _FILL_

          # Get the accuracy
          accuracy_hist_test += is_correct

      # Normalize the metrics by the right number
      accuracy_hist_test /= len(test_dl.dataset)
      loss_hist_test /= len(test_dl.dataset)
      print(f'Test Metrics Epoch {epoch} Loss {loss_hist_test:.4f} Accuracy {accuracy_hist_test:.4f}')